In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pickle

Basic Block: Conv → BN → ReLU → Conv → BN → Add skip → ReLU

In [8]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=2, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)
        self.batchnorm1 = nn.BatchNorm1d(out_channels)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size)
        self.batchnorm2 = nn.BatchNorm1d(out_channels)
        # add skip
        self.downsample = downsample
        self.relu2 = nn.ReLU(inplace=True)


    def forward(self, x):
        ip = x
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)
        
        if self.downsample is not None:
            ip = self.downsample(ip)
        x = x + ip # add the skipped input
        
        x = self.relu2(x)

        return x   
    

In [3]:
with open(r"C:\Users\tom_r\Desktop\Machine-Learning\RF_spectrum_analysis\subset.pkl", 'rb') as file:
    dataset = pickle.load(file)

In [4]:
for k, v in dataset.items():
    print(f"{k}: shape {v.shape}, dtype {v.dtype}")

X: shape (532480, 1024, 2), dtype float16
y: shape (532480,), dtype int64
snr: shape (532480,), dtype int64


In [5]:
unique_labels = np.unique(dataset['y'])
label_map = {old: new for new, old in enumerate(unique_labels)}

# Remap y labels
mapped_y = np.vectorize(label_map.get)(dataset['y'])

In [6]:
X_tensor = torch.tensor(dataset['X'], dtype=torch.float32)  
X_tensor = X_tensor.permute(0, 2, 1)  # (N, 2, 1024)

In [15]:
downsample = nn.Sequential(
    nn.Conv1d(in_channels=2, out_channels=64, kernel_size=1, stride=2),
    nn.BatchNorm1d(64)
)

basic = BasicBlock(in_channels=2, out_channels=64, downsample=downsample)


output = basic.forward(X_tensor[0].unsqueeze(0)) 
output.shape

RuntimeError: The size of tensor a (509) must match the size of tensor b (512) at non-singleton dimension 2